In [8]:
#normalize X values to help model converge
def normalize(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    data = (data - mean) / std
    return data

In [9]:
import lightkurve as lk
import pandas as pd
import numpy as np
from tsfresh import extract_features
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters

pixelfile = lk.search_targetpixelfile("Trappist-1")[1].download()
lc = pixelfile.to_lightcurve(method="pld")
matrix = np.zeros((2,3197))
matrix[0] = np.array(lc.flux)[:3197]
ts = lk.read("Lightcurves/HAT-P-7b-lc.fits")
lc = lk.LightCurve(ts)
matrix[1] = np.array(lc.flux)[:3197]
inference = normalize(matrix)

In [14]:
flux_copy = np.array(lc.flux)
amt_pad = 3197-(len(lc.flux)%3197)
print(len(flux_copy), amt_pad)
print(len(flux_copy) + amt_pad)
print(type(flux_copy))
flux_median = np.full(shape=amt_pad,fill_value=np.median(np.array(lc.flux)))
flux_copy = np.append(flux_copy,flux_median)
print(len(flux_copy))
tim_size = len(flux_copy)//3197
print(tim_size)
tim_matrix = np.reshape(flux_copy, (tim_size,3197))
print(tim_matrix)

matr = pd.DataFrame({"id" : np.zeros(len(lc.flux), dtype=int),
                "time" : list(range(0, len(lc.flux))),
                "flux" : np.array(lc.flux).byteswap().newbyteorder()})
print(matr)

18424 758
19182
<class 'numpy.ndarray'>
19182
6
[[15093.22  15069.889 15082.259 ... 15098.509 15102.568 15069.43 ]
 [15047.939 15085.095 15090.239 ... 15109.166 15079.969 15102.488]
 [15085.565 15103.742 15087.44  ... 15095.961 15081.653 15133.081]
 [15092.11  15091.476 15077.059 ... 15083.673 15113.691 15091.122]
 [15077.417 15079.411 15085.794 ... 15002.069 14969.357 14988.887]
 [14973.506 14973.703 14993.971 ... 15085.748 15085.748 15085.748]]
       id   time          flux
0       0      0  15093.219727
1       0      1  15069.888672
2       0      2  15082.258789
3       0      3  15110.080078
4       0      4  15098.575195
...    ..    ...           ...
18419   0  18419  15101.319336
18420   0  18420  15103.712891
18421   0  18421  15087.833984
18422   0  18422  15135.316406
18423   0  18423  15078.958008

[18424 rows x 3 columns]


In [15]:
extracted_features = extract_features(matr, column_id= "id", column_sort= "time", 
                                      column_value= "flux", 
                                      default_fc_parameters= EfficientFCParameters())
extracted_features

Feature Extraction: 100%|██████████| 1/1 [00:14<00:00, 14.26s/it]


,flux__variance_larger_than_standard_deviation,flux__has_duplicate_max,flux__has_duplicate_min,flux__has_duplicate,flux__sum_values,flux__abs_energy,flux__mean_abs_change,flux__mean_change,flux__mean_second_derivative_central,flux__median,...,flux__permutation_entropy__dimension_5__tau_1,flux__permutation_entropy__dimension_6__tau_1,flux__permutation_entropy__dimension_7__tau_1,flux__query_similarity_count__query_None__threshold_0.0,"flux__matrix_profile__feature_""min""__threshold_0.98","flux__matrix_profile__feature_""max""__threshold_0.98","flux__matrix_profile__feature_""mean""__threshold_0.98","flux__matrix_profile__feature_""median""__threshold_0.98","flux__matrix_profile__feature_""25""__threshold_0.98","flux__matrix_profile__feature_""75""__threshold_0.98"
0,1.0,0.0,0.0,1.0,277856320.0,4.190426e+12,19.163937,-0.000774,-0.000896,15085.748047,...,4.784825,6.559999,8.378373,NaN,0.96803,2.484583,1.941508,1.965132,1.816393,2.091379


In [3]:
with open('Models/lk.npy', 'wb') as f:
    np.save(f, inference)

5.76290272130122
